In [1]:
import os
import pickle
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt
from calculate_score import *
from collections import defaultdict

def load_pickle(filepath):
    with open(filepath, "rb") as f:
        json_data = pickle.load(f)
    return json_data

In [ ]:
marker_dict = {9: "left_wrist", 10: "right_wrist", 
                15: "left_ankle", 16: "right_ankle", 
                }   # 11: "left_hip",12: "right_hip"

config_hand1 = {"a": 60, "b": 140, "sub_dir": "hand", "markerA_id": [9, 10], 
                "modes": ["zero_uni", "zero_bi"], "metric": ["pos", "vel"]}

config_foot1 = {"a": 60, "b": 140, "sub_dir": "foot", "markerA_id": [15, 16], 
                "modes": ["zero_uni", "zero_bi"], "metric": ["pos", "vel"]}


tol = 6
tol_type = "rel"        # rel: relative, abs" absolute
configs = [config_hand1, config_foot1]

for cfg in configs:
    a = cfg["a"]
    b = cfg["b"]
    sub_dir = cfg["sub_dir"]
    for markerA_id in cfg["markerA_id"]:
        for mode in cfg["modes"]:
            for metric in cfg["metric"]:
                
                read_path = f"./one_sensor/tempo_{a}_{b}/{metric}/{sub_dir}/{marker_dict[markerA_id]}_{mode}_{a}_{b}.pkl"
                save_path = f'./one_sensor/tempo_{a}_{b}/{metric}/score/{sub_dir}/{marker_dict[markerA_id]}_{mode}_{a}_{b}_score.pkl'
                calc_score(mode, read_path, save_path, tol_type, tol)

#### New Update March 12

In [ ]:
metric = "pos"
tol = 6
tol_type = "rel"
mode = "zero_uni"

a = 60; b =140
# segment_keys = ["both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y"]

# for seg_key in segment_keys:
#     read_file = f"./one_sensor/tempo_{a}_{b}/{metric}/{seg_key}_{mode}_{a}_{b}.pkl"
#     save_file = f"./one_sensor/tempo_{a}_{b}/{metric}/{seg_key}_{mode}_{a}_{b}_score.pkl"
#     updated_calc_score(mode, read_file, save_file, tol_type, tol)

In [ ]:
def calculate_metrics(ref, calculated, tol_type="abs", tolerance=5, mode = "one"):
    """
    Calculate Acc1, Acc2, Metric3, OE1, and OE2.
    Now, 'calculated' is a list of tuples, each containing two candidate BPM values.
    For each index, we select the candidate BPM that is closest to the ground truth.
    
    Args:
        ref (np.ndarray): Reference BPMs.
        calculated (list of tuple): Each tuple contains two candidate BPMs.
        tol_type (str): "abs" or "rel" tolerance type.
        tolerance (float): Tolerance value.
        
    Returns:
        dict: Dictionary with accuracy and error metrics.
    """
    # Determine the tolerance for comparison
    if tol_type == "rel":
        ref_tolerance = ref * (tolerance / 100)
    elif tol_type == "abs":
        ref_tolerance = tolerance

    if mode == "many":
        # For each index, choose the candidate BPM that minimizes the error from ref[i]
        chosen = np.array([
            min(cand, key=lambda bpm: abs(bpm - ref[i]))
            for i, cand in enumerate(calculated)
        ])
    
    elif mode == "one":
        chosen = calculated
    
    # Compute accuracy metrics using the chosen BPMs
    acc1_count = np.sum(np.abs(chosen - ref) <= ref_tolerance)
    acc2_count = np.sum(
        (np.abs(chosen - ref) <= ref_tolerance) |
        (np.abs(chosen - 2 * ref) <= ref_tolerance) |
        (np.abs(chosen - ref / 2) <= ref_tolerance)
    )
    
    scales = [1, 2, 0.5, 3, 1/3]
    acc3_count = np.sum(np.any([np.abs(chosen - ref * scale) <= ref_tolerance for scale in scales], axis=0))
    
    hits_idx = np.where(np.abs(chosen - ref) <= ref_tolerance)[0]
    hits_dbl_idx = np.where(np.abs(chosen - 2 * ref) <= ref_tolerance)[0]
    hits_hf_idx = np.where(np.abs(chosen - ref / 2) <= ref_tolerance)[0]
    
    bpm_hits_idx = chosen[hits_idx]
    bpm_hits_dbl_idx = chosen[hits_dbl_idx]
    bpm_hits_hf_idx = chosen[hits_hf_idx]
    
    base_tempo = ref[hits_idx]
    
    # Overestimated errors:
    oe1_count = np.sum(
        (chosen > ref) &
        ~np.any([np.abs(chosen - ref * scale) <= ref_tolerance for scale in scales], axis=0)
    )
    oe2_count = np.sum(
        (chosen > ref) &
        np.any([np.abs(chosen - ref * scale) <= ref_tolerance for scale in scales], axis=0)
    )
    
    total = len(ref)
    
    metrics = {
        "acc1": (acc1_count / total) * 100,
        "acc2": (acc2_count / total) * 100,
        "acc3": (acc3_count / total) * 100,
        "OE1": (oe1_count / total) * 100,
        "OE2": (oe2_count / total) * 100,
        "base_tempo": base_tempo,
        "hits_idx": hits_idx,
        "hits_dbl_idx": hits_dbl_idx,
        "hits_hf_idx": hits_hf_idx,
        "bpm_hits_idx": bpm_hits_idx,
        "bpm_hits_dbl_idx": bpm_hits_dbl_idx,
        "bpm_hits_hf_idx": bpm_hits_hf_idx
    }
    return metrics

#### Multi bpm

In [ ]:
def estimate_tempo_pair(a, b, mode ):

    segment_pairs = [
        ("both_hand_x", "both_foot_x"),                     # bpm_pair1 35 % | bpm_pair2 55%
        ("both_hand_y", "both_foot_y"),                     # bpm_pair1 70 % | bpm_pair2 81%
        ("both_hand_resultant", "both_foot_resultant"),     # bpm_pair1 49 % | bpm_pair2 66%
        ("bothhand_xy", "bothfoot_xy"),                     # bpm_pair1 55 % | bpm_pair2 68%
        ]

    json_data = {"both_hand_foot_x": {"bpm_pair": None, "bpm_quad": None, "Acc1_bpm_pair": None, "Acc1_bpm_quad": None}, 
                 "both_hand_foot_y": {"bpm_pair": None, "bpm_quad": None, "Acc1_bpm_pair": None, "Acc1_bpm_quad": None}, 
                 "both_hand_foot_resultant": {"bpm_pair": None, "bpm_quad": None, "Acc1_bpm_pair": None, "Acc1_bpm_quad": None}, 
                 "both_hand_foot_xy": {"bpm_pair": None, "bpm_quad": None, "Acc1_bpm_pair": None, "Acc1_bpm_quad": None}, }
    
    for hnd, ft in segment_pairs:

        read_file1 = f"./one_sensor/tempo_{a}_{b}/pos/{hnd}_{mode}_{a}_{b}.pkl"
        read_file2 = f"./one_sensor/tempo_{a}_{b}/pos/{ft}_{mode}_{a}_{b}.pkl"

        read_file3 = f"./one_sensor/tempo_{a}_{b}/vel/{hnd}_{mode}_{a}_{b}.pkl"
        read_file4 = f"./one_sensor/tempo_{a}_{b}/vel/{ft}_{mode}_{a}_{b}.pkl"

        df1 = pd.read_pickle(read_file1)
        df2 = pd.read_pickle(read_file2)
        df3 = pd.read_pickle(read_file3)
        df4 = pd.read_pickle(read_file4)

        bpm_pair = []
        bpm_quad = []
        for n in range(df1.shape[0]):
            bpm1 = df1.iloc[n]["bpm_median"]    # hand bpm
            bpm2 = df2.iloc[n]["bpm_median"]    # foot bpm
            
            bpm3 = df3.iloc[n]["bpm_median"]    # hand bpm
            bpm4 = df4.iloc[n]["bpm_median"]    # foot bpm
            
            bpm_pair.append((bpm1, bpm2))
            bpm_quad.append((bpm1, bpm2, bpm3, bpm4))
            
        ref = df1["music_tempo"].to_numpy()
        
        metrics1 = calculate_metrics(ref, bpm_pair, tol_type = "rel", tolerance = 6, mode = "many")
        metrics2 = calculate_metrics(ref, bpm_quad, tol_type = "rel", tolerance = 6, mode = "many")
        
        print(f"{hnd} | {ft}:", round(metrics1["acc1"]), round(metrics2["acc1"]))
    
    return    

tempo_cases = {
    
}


both_hand_x | both_foot_x: 35 55
both_hand_y | both_foot_y: 70 82
both_hand_resultant | both_foot_resultant: 49 67
bothhand_xy | bothfoot_xy: 55 69


In [ ]:
def estimate_tempo_pair(a, b, mode):
    # Define the segment pairs to process
    segment_pairs = [
        ("both_hand_x", "both_foot_x"),                     # Example: bpm_pair ~35%, bpm_quad ~55%
        ("both_hand_y", "both_foot_y"),                     # Example: bpm_pair ~70%, bpm_quad ~81%
        ("both_hand_resultant", "both_foot_resultant"),     # Example: bpm_pair ~49%, bpm_quad ~66%
        ("bothhand_xy", "bothfoot_xy"),                     # Example: bpm_pair ~55%, bpm_quad ~68%
    ]
    
    # Prepare a dictionary to hold results for each segment pair
    json_data = {}
    
    for hnd, ft in segment_pairs:
        # Build file paths for position and velocity data
        read_file1 = f"./one_sensor/tempo_{a}_{b}/pos/{hnd}_{mode}_{a}_{b}.pkl"
        read_file2 = f"./one_sensor/tempo_{a}_{b}/pos/{ft}_{mode}_{a}_{b}.pkl"
        read_file3 = f"./one_sensor/tempo_{a}_{b}/vel/{hnd}_{mode}_{a}_{b}.pkl"
        read_file4 = f"./one_sensor/tempo_{a}_{b}/vel/{ft}_{mode}_{a}_{b}.pkl"
        
        # Load the dataframes
        df1 = pd.read_pickle(read_file1)
        df2 = pd.read_pickle(read_file2)
        df3 = pd.read_pickle(read_file3)
        df4 = pd.read_pickle(read_file4)
        
        # Build candidate BPM pairs (for positions) and quads (for positions and velocities)
        bpm_pair = []
        bpm_quad = []
        for n in range(df1.shape[0]):
            bpm1 = df1.iloc[n]["bpm_median"]   # hand (position)
            bpm2 = df2.iloc[n]["bpm_median"]   # foot (position)
            bpm3 = df3.iloc[n]["bpm_median"]   # hand (velocity)
            bpm4 = df4.iloc[n]["bpm_median"]   # foot (velocity)
            bpm_pair.append((bpm1, bpm2))
            bpm_quad.append((bpm1, bpm2, bpm3, bpm4))
        
        # Use music_tempo from df1 as the reference tempo array.
        ref = df1["music_tempo"].to_numpy()
        
        # Calculate metrics for the candidate pairs and quads
        metrics1 = calculate_metrics(ref, bpm_pair, tol_type="rel", tolerance=6, mode = "many")
        metrics2 = calculate_metrics(ref, bpm_quad, tol_type="rel", tolerance=6, mode = "many")
        
        # For clarity, build a key for the current segment pair
        pair_key = f"{hnd}_{ft}"
        
        # Print the Acc1 results for each candidate method
        print(f"{pair_key}: Acc1 pair = {round(metrics1['acc1'])}%, Acc1 quad = {round(metrics2['acc1'])}%")
        
        # Store the candidate BPMs and accuracy metrics in json_data
        json_data[pair_key] = {
            "bpm_pair": bpm_pair,
            "bpm_quad": bpm_quad,
            "Acc1_bpm_pair": metrics1["acc1"],
            "Acc1_bpm_quad": metrics2["acc1"]
        }
    
    return json_data


a = 60; b=130
mode = "zero_uni"

jdata = estimate_tempo_pair(a, b, mode)

### One bpm

In [ ]:
segment_keys = [
                "both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y",
                "bothhand_xy", "bothfoot_xy","both_hand_resultant", "both_foot_resultant",
                
                "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy",   # singular | 35, 40, 34, 36
               
                "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y",   # singular
                "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y",   # singular
                 
                "left_hand_resultant", "right_hand_resultant", "left_foot_resultant", "right_foot_resultant",   # singular | 18,20,17,17 %
                ] 

for idx, f_name in enumerate(segment_keys):
    f_path = f"./one_sensor/tempo_{a}_{b}/pos/{f_name}_{mode}_{a}_{b}.pkl"
    df_ax = pd.read_pickle(f_path)
    
    ref = df_ax["music_tempo"].to_numpy()
    metrics = calculate_metrics(ref, np.asarray(df_ax["bpm_median"]), tol_type = "rel", tolerance = 6, mode = "one")
    print(f"{f_name}:", round(metrics["acc1"]))
    
    if (idx+1)%4 == 0:
        print("\n")

both_hand_x: 21
both_hand_y: 53
both_foot_x: 18
both_foot_y: 53


bothhand_xy: 37
bothfoot_xy: 37
both_hand_resultant: 29
both_foot_resultant: 32


lefthand_xy: 36
righthand_xy: 40
leftfoot_xy: 34
rightfoot_xy: 36


left_hand_x: 28
right_hand_x: 28
left_hand_y: 28
right_hand_y: 28


left_foot_x: 29
right_foot_x: 29
left_foot_y: 41
right_foot_y: 41


left_hand_resultant: 19
right_hand_resultant: 20
left_foot_resultant: 17
right_foot_resultant: 18


